# CentraleSupelec - Natural language processing
# Practical session n°7

## Natural Language Inferencing (NLI): 

(NLI) is a classical NLP (Natural Language Processing) problem that involves taking two sentences (the premise and the hypothesis ), and deciding how they are related- if the premise *entails* the hypothesis, *contradicts* it, or *neither*.

Ex: 


| Premise | Label | Hypothesis |
| --- | --- | --- |
| A man inspects the uniform of a figure in some East Asian country. | contradiction | The man is sleeping. |
| An older and younger man smiling. | neutral | Two men are smiling and laughing at the cats playing on the floor. |
| A soccer game with multiple males playing. | entailment | Some men are playing a sport. |

### Stanford NLI (SNLI) corpus

In this labwork, I propose to use the Stanford NLI (SNLI) corpus ( https://nlp.stanford.edu/projects/snli/ ), available in the *Datasets* library by Huggingface.

    from datasets import load_dataset
    snli = load_dataset("snli")
    #Removing sentence pairs with no label (-1)
    snli = snli.filter(lambda example: example['label'] != -1) 

## Subject

You are asked to provide an operational Jupyter notebook that performs the task of NLI. For that, you need to tackle the following aspects of the problem:

1. Loading and preprocessing the data
2. Designing a PyTorch model that, given two sentences, decides how they are related (*entails*, *contradicts* or *neither*.)
3. Training and evaluating the model using appropriate metrics
4. (Optional) Allowing to play with the model (forward user sentences and visualize the prediction easily)
5. (Optional) Providing visual insight about the model (i.e. visualizing the attention if your model is using attention)

Although it is not mandatory, I suggest that you use a transformer model to perform the task. For that, you can use the *Transformer* library by Huggingface.

## Evaluation

The evaluation will be based on several criteria:

- Clarity and readability of the notebook. The notebook is the report of you project. Make it easy and pleasant to read.
- Justification of implementation choices (i.e. the network, the cost funtion, the optimizer, ...)
- Quality of the code. The various deeplearning and NLP labworks provide many example of good practices for designing experiments with neural networks. Use them as inspirational examples!

## Additional recommendations

- You are not seeking to publish a research paper! I'm not expecting state-of-the-art results! The idea of this labwork is to assess that you have integrated the skills necessary to handle textual data using deep neural network techniques.

- This labwork will be evaluated but we are still here to help you! Don't hesitate to request our help if you are stuck.

- If you intend to use BERT based models, let me give you an advice. The bert-base-* models available in *Transformers* need more than 12Go to be fine-tuned on GPU. To avoid memory issues, you can use several solutions: 

    - Use a lighter BERT based model such as DistilBERT, ALBERT, ...
    - Train a classification model on top of BERT, whithout fine-tuning it (i.e. freezing BERT weights)

## Huggingface documentations

In case you want to use the huggingface *Datasets* and *Transformer* libraries (which I advice), here are some useful documentation pages:

- Dataset quick tour

    https://huggingface.co/docs/datasets/quicktour.html
    
- Documentation on data preprocessing for transformers

    https://huggingface.co/transformers/preprocessing.html
    
- Transformer Quick tour (with distilbert example for classification).

    https://huggingface.co/transformers/quicktour.html
    


In [1]:
from datasets import load_dataset
snli = load_dataset("snli")
#Removing sentence pairs with no label (-1)
snli = snli.filter(lambda example: example['label'] != -1) 

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/938 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.9M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.26M [00:00<?, ?B/s]

Dataset snli downloaded and prepared to /usr/users/gpusdi1/gpusdi1_15/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c. Subsequent calls will reuse this data.


  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/551 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

## Roadmap
- dataset transformer
- model
- train/test
- metrics ?
- Modelcheckpoint, loss/accuracy graph
- example of classification and visualisation insight attention

In [18]:
!pip3 install --user transformers

  Using cached https://files.pythonhosted.org/packages/ed/d5/f4157a376b8a79489a76ce6cfe147f4f3be1e029b7144fa7b8432e8acb26/transformers-4.4.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fd/5b/44baae602e0a30bcc53fbdbc60bd940c15e143d252d658dfdefce736ece5/tokenizers-0.10.1-cp36-cp36m-manylinux2010_x86_64.whl
Processing /usr/users/gpusdi1/gpusdi1_15/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45/sacremoses-0.0.43-cp36-none-any.whl
You should consider upgrading via the 'pip install --upgrade pip' command.


In [19]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel

In [21]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [69]:
test_dataset = snli["test"]
train_dataset = snli["train"]
validation_dataset = snli["validation"]
for data in test_dataset:
    print(data)
    break

{'hypothesis': 'The church has cracks in the ceiling.', 'label': 1, 'premise': 'This church choir sings to the masses as they sing joyous songs from the book at a church.'}


In [105]:
def tokenized_dataset(dataset, tokenizer):
    dataset_tokenized = []
    sentences_tokenized = tokenizer(dataset["hypothesis"],dataset["premise"], padding=True, return_tensors="pt")
    return [{
            "input_ids": sentence_tokenized["input_ids"],
            "attention_mask": sentence_tokenized["attention_mask"],
            
            "label": torch.tensor(label)
        } for sentence_tokenized, label in zip(sentences_tokenized,dataset["label"])]

In [104]:
config = {
    "BATCH_SIZE": 32
}

In [64]:
class DatasetTransformer(torch.utils.data.Dataset):

    def __init__(self, base_dataset, transform):
        self.base_dataset = base_dataset
        self.transform = transform

    def __getitem__(self, index):
        item = self.base_dataset[index]
        hypothesis = item["hypothesis"]
        label = item["label"]
        premise = item["premise"]
        sentence = hypothesis + " " + premise
        sentence_tokenized = self.transform(hyphotesis, return_tensors="pt")
        return {
            "sentence": sentence_tokenized,
            "label": torch.tensor(label)
        }
        

    def __len__(self):
        return len(self.base_dataset)

In [106]:
test_dataset_tokenized = tokenized_dataset(test_dataset, tokenizer)
train_dataset_tokenized = tokenized_dataset(train_dataset, tokenizer)
validation_dataset_tokenized = tokenized_dataset(validation_dataset, tokenizer)

In [107]:
#test_dataset_transformer  = DatasetTransformer(test_dataset , tokenizer)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset_tokenized,
                                              batch_size=config["BATCH_SIZE"],
                                              shuffle=False)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset_tokenized,
                                              batch_size=config["BATCH_SIZE"],
                                              shuffle=False)
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset_tokenized,
                                              batch_size=config["BATCH_SIZE"],
                                              shuffle=False)

In [108]:
for data in enumerate(train_loader):
    print(data)
    break

(0, {'sentence': ['input_ids', 'attention_mask'], 'label': tensor([1, 2])})


In [ ]:
class Model():
    def __init__(self, num_classes):
        super(Model, self).__init__(model_name="distilbert-base-uncased")
        self.model = DistilBertModel.from_pretrained(model_name)
        self.fc = 0

In [93]:
from transformers import DistilBertTokenizer, DistilBertForMultipleChoice
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
model = DistilBertForMultipleChoice.from_pretrained('distilbert-base-cased')
prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
choice0 = "It is eaten with a fork and a knife."
choice1 = "It is eaten while held in the hand."
labels = torch.tensor([[0], [1]]).unsqueeze(0)  # choice0 is correct (according to Wikipedia ;)), batch size 1
print(labels)
encoding = tokenizer([[prompt, choice0], [prompt, choice1]], return_tensors='pt', padding=True)
print(encoding)
outputs = model(**{k: v.unsqueeze(0) for k,v in encoding.items()}, labels=labels) # batch size is 1
loss = outputs.loss
logits = outputs.logits
print(loss)
print(logits)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForMultipleChoice: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForMultipleChoice were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You

tensor([[[0],
         [1]]])
{'input_ids': tensor([[  101,  1130,  2413,   117, 13473,  1462,  1107,  4698, 11106,   117,
          1216,  1112,  1120,   170,  4382,   117,  1110,  2756,  8362,  1116,
         11815,  1181,   119,   102,  1135,  1110,  8527,  1114,   170, 13097,
          1105,   170,  4937,   119,   102],
        [  101,  1130,  2413,   117, 13473,  1462,  1107,  4698, 11106,   117,
          1216,  1112,  1120,   170,  4382,   117,  1110,  2756,  8362,  1116,
         11815,  1181,   119,   102,  1135,  1110,  8527,  1229,  1316,  1107,
          1103,  1289,   119,   102,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])}


RuntimeError: 1D target tensor expected, multi-target not supported